#### **Self-Attention**

The whole idea of transformers and its related architecutures are based on the "self-attention" operation.<br><br><br>
Self-attention is a sequence-to-sequence operation: a sequence of vector goes in, and a sequence of vector comes out. Let's input vectors $\mathbf{x_1, x_2, x_3,...,x_t}$ and output vectors $\mathbf{y_1, y_2, y_3,..., y_t}$ and all vectors have $\mathbb{k}$ dimensions.<br>
To produce output vector $\mathbf{y_i}$, the self attention operation simply takes a weighted average over all the input vectors<br><br>
$$\mathbf{y_i} = \sum_j \mathbf{w}_{ij}\mathbf{x}_j$$
where $j$ indexes over the whole sequence and the weights sum to one over all $j$. The weight $\mathbf{w_{ij}}$ is not a parameter, as in a normal neural net, but it is derived from a function over $\mathbf{x_i}$ and $\mathbf{x_j}$. The simplest option for this function is the dot product:
$$\mathbf{w}_{ij}{'} = \mathbf{x}_i^T\mathbf{x}_j$$
Note that $\mathbf{x_i}$ is the input vector at the same position as the current output vector $\mathbf{y_i}$. For the next output vector, we get an entirely new series of dot products, and a different weighted sum.<br><br>
The dot product gives us a value anywhere between negative and positive infinity, so we apply a softmax to map the values to $[0, 1]$ and to ensure tht they sum to $1$ over the whole sequence:
$$\mathbf{w}_{ij} = \frac{\exp  \mathbf{w}_{ij}{'} }{\sum_j \exp \mathbf{w}_{ij}'}$$

Assigning embeddings vectors $v_t$ to each word $t$ in our vocabulory is known as *embedding layer*, the values of the embedding vectors will be learned. It turns the word sequence *the, cat, walks, on, the, street* into vector space
$$\mathbf{v}_{the}, \mathbf{v}_{cat}, \mathbf{v}_{walks}, \mathbf{v}_{on}, \mathbf{v}_{the}, \mathbf{v}_{street}$$
When this sequence is fed into the self-attention layer, the output is another sequence of vectors<br>
$$\mathbf{y}_{the}, \mathbf{y}_{cat}, \mathbf{y}_{walks}, \mathbf{y}_{on}, \mathbf{y}_{the}, \mathbf{y}_{street}$$
where $\mathbf{y}_{cat}$ is a weighted sum over all the embedding vectors in the first sequence, weighted by their(normalize) dot-product with $\mathbf{v}_{cat}$.<br><br><br>

Since we are *learning* what the values in $\mathbf{v}_t$ should be, how "related" two words are is entirely determined by the task. In most case, the definite articles *the* is not very relevant to the interpretation of the other words in the sentence; therefor we will likely end up with an embedding $\mathbf{v}_{the}$ that has a low or negative dot product with all other words. On the other hand, to interpret what *walks* means in this sentence, it's very helpful to work out *who* is doing the walking. This is likely expressed by a noun, so for nounds like *cat* and verbs like *walks*, we will likely learn embeddings $\mathbf{v}_{cat}$ and $\mathbf{v}_{walks}$ that have a high, positive dot product together.<br><br><br>
This is the basic intuition behind self-attention. The dot product expresses how related two vectors in the input seqeunce are, with "related" defined by the learning task, and the output vectors are weighted sums over the whhoel input sequence, with the weights determined by these dot products.<br><br><br>
Before we move on, it's worthwhile to note the following properties, which are unsual for a sequence-to-sequence operation:
*   There are no parameters(yet). What the basic self-attention actually does is entirely determined by whatever mechanism creates the input sequence. Upstream mechanisms, like an embedding layer, drive the self-attention by learning representaions with particular dot products (although we'll add a few parameters later).
*    Self attention sees its input as a set, not a sequence. If we permute the input sequence, the output sequence will be exactly the same, except permuted also (i.e. self-attention is permutation equivariant). We will mitigate this somewhat when we build the full transformer, but the self-attention by itself actually ignores the sequential nature of the input



The whole self-attention equation can be written as:
$$\begin{align*}
\mathbf{y}_i = \displaystyle \sum_j \left\{ \frac{\exp \mathbf{x}_i^T \mathbf{x}_k}{\sum_k \exp \mathbf{x}_i^T \mathbf{x}_k} \right\} \mathbf{x}_j
\end{align*}$$<br><br>
$k, j$ are both same, indexes over the same sequence. <br><br>This implies, when the sentences is given to the self-attention, in the form of sequence of input vectors and corresponding output vectors are generated. The corresponding output vector is sum of probability of each input vector multiplied(hadamard/dot product) with actual input vector. The probabilities are generated by dot product of corresponding input vector(transposed) with all other input vectors tokens. The way input vectors are arranged is learned by the self-attention.

#### **In Pytorch: Basic Self-Attention**


Let the input be a sequence of $\mathbf{t}$ vectors with $\mathbf{k}$ dimensions as $\mathbf{t}$ by $\mathbf{k}$ matrix $\mathbf{X}$. Including a minibatch dimension b, gives us an input tensor of size ($\mathbf{b, t, k}$). <br><br>

The set of all dor products for $\mathbf{w}_{ij}^{'} $ forms a matrix, which we can be computed simply by multiplying $\mathbf{X}$ by its transpose


```python
import torch
import torch.nn.functional as F

# assume we have some tensor x with size (b, t, k)
x = ...

raw_weights = torch.bmm(x, x.transpose(1,2))
# - torch.bmm is a batched matrix mulitplication. It
# applies matrix multiplication over batches of Matrices
```



Then, to turn the raw weights $\mathbf{w}_{ij}^{'}$ into positive values tht sum to one, we apply a row-wise softmax:

```python
weights = F.softmax(raw_weights, dim=2)
```
Finally, to compute the output sequence, we just multiply the weight matrix by $\mathbf{X}$. this results in a batch of output matrices $\mathbf{Y}$ of size ($\mathbf{b, t, k}$) whose rows are weighted sums over the rows of $\mathbf{X}$.


```python
y = torch.bmm(weights, x)
```
basic self-attention consists of two matrix multiplications & one softmax.

### **Additional Tricks**

####**Query, Keys and Values**


The input $\mathbf{x}_i$ is used in threee(3) different ways:<br>
1.  It is compared with every other vector to create the weights for its own output $\mathbf{y}_i$, it is called as **query** $\implies$ $\mathbf{q}_i = \mathbf{W}_q \mathbf{x}_i$
2.  It is compared with every other vector to create the weights for the outputof the $j$-th vector $\mathbf{y}_j$, it is called as **key** $\implies$$\mathbf{k}_i = \mathbf{W}_k \mathbf{x}_i$
3.  It is used as part of the weighted sum to compute each output vector once the weights have been created, it is called as **value** $\implies$ $\mathbf{v}_i = \mathbf{W}_v \mathbf{x}_i$

$$\mathbf{q}_i = \mathbf{W}_q \mathbf{x}_i \qquad \mathbf{k}_i = \mathbf{W}_k \mathbf{x}_i \qquad \mathbf{v}_i = \mathbf{W}_v \mathbf{x}_i\\
\mathbf{w}_{ij}^{'} = \mathbf{q}_i^T\mathbf{k}_j\\
\mathbf{w}_{ij} = \text{softmax}(\mathbf{w}_{ij}^{'})\\
\mathbf{y}_i = \sum_j \mathbf{w}_{ij} \mathbf{v}_j$$<br><br>

####**Scaling the dot product**


To stop the softmax gradients from growing too large and slow down the learning. The average value of the dot product grows with embedding dimentions k, it helps to scale down the growing softmax function
$$\mathbf{w}_{ij}^{'} = \dfrac{\mathbf{q}_i^T \mathbf{k}_j}{\sqrt{k}}$$
why k? its Euclidean length take from $\mathbb{R}^k$

####**Multi-head attention**


A word can mean different things to different neighbours. "*The quick brown fox jumps over the lazy dog*". We see word "*jumps*" had different relations to different parts of the sentence. "*fox*" express who doing the action and "*dog*" represents an animate object that was beneath action was performed. <br><br>
The all the information is summed together in the single-attention. The inputs $\mathbf{x}_{fox}$ and $\mathbf{x}_{dog}$ can influence the output $\mathbf{y}_{jump}$ by different amounts, depending on their dot-product with $\mathbf{x}_{jump} $ but they can't influence it in different ways. If we want the similar different information, we can achieve this by giving combining several self-attention mechanisms (which we'll index with $r$), each with different matrices $\mathbf{W}_q^r, \mathbf{W}_k^r, \mathbf{W}_v^r$. These are called as *attention heads.*<br><br>
For input $\mathbf{x}_i$ each attention head produces a different output vector $\mathbf{y}_i^r$. We concatenate these, and pass them through a linear transformation to reduce the dimension back to $k$.<br><br>
The simplest way to understand multi-head self-attention is to see it as a small number of copies of the self-attention mechanisam applied in parallel, each with their own key, value and query transformation. This works well, but for $\mathbf{R}$ heads, the self-attention operation is $\mathbf{R}$ times as slow. There is a way to implement multi-head self-attention so that it is roughly as fast as the single-head version, but we still get the benefit of having different self-attention operations in parallel. To accomlish this, each head receives low-dimensinal keys, queries and values. If the input vecor has $k = 256$ dimensions, and we have $h = 4$ attention heads, we multiply the input vectors by a $256 \times 64$ matrix to project them down to a sequence of $64$ dimensional vectors. For every head, we do this 3 times: for the keys, the queries and the values.<br><br>

This requires $3h$ matrices of size $k$ by $k/h$. In total, this gives us $3hk \dfrac{k}{h} = 3k^2$ parameters to compute the inputs to the multi-head self-attention; the same as we had for the single-head self-attention. We can even implement this with just three $ k \times k$ matrix multiplications as in the single-head self-attention. the only extra operation we need is to slice the resulting sequence of vectors into chunks

#### **In Pytorch: complete self-attention**

Let's implement a self-attention module using Pytorch module<br>

```python
# import libraries
import torch
from torch import nn
import torch.nn.fuctional as F

class SelfAttention(nn.Module):
    def __init__(self, k, heads=4, mask=False):

        super().__init__()

        assert k % heads == 0

        self.k, self.heads = k, heads
```
Assert imposes that embeddings dimensions should be divisible by the number of heads. We will setup some linear transformations with emb by emb matrices. The "nn.Linear" module with diables bias gives us such a projection


```python
# These compute the queries, keys and vlaues for all
    # heads

    self.tokeys = nn.Linear(k, k, bias=False)
    self.toqueries = nn.Linear(k, k, bias=False)
    self.tovlaues = nn.Linear(k, k, bias=False)

    # This will be applied after the multi-head self-attention operation
    self.unifyheads = nn.Linear(k, k)

```
Lets compute queries, keys, and values for all heads as the part of forward function and also implements the self-attention


```python
    #
    # forward function
    def forward(self, x):
        b, t, k = x.size()
        h = self.heads

        queries = self.toqueries(x)
        keys = self.tokeys(x)
        values = self.tovalues(x)
```
This gives us three vectors sequence of the full embedding dimensions $k$. As we saw above we can now cut these into $h$ chunks.


```python
        #
        s = k // h

        keys = keys.view(b, t, h, s)
        queries = queries.view(b, t, h, s)
        values = values.view(b, t, h, s)
```






This simply reshapes the tensors to add a dimension that iterate over the heads. For a singel vector in our sequence we can think of it as reshaping a vector of dimensions $k$ into a matrix of $h$ by $k // h$.
We need to compute the dor products. This is the same operation for every head, so we fold the heads into the batch dimensions. This ensures that we can use torch.bmm() as before, and the whole collection of keys, queries and values will just be seen as a slightly larger batch.<br><br>
Since the head and batch dimension are not next to each other, we need to transpose before we reshape.(This is costly, ut it seems to be unavoidable.)


```python
        #
        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contigous().view(b * h, t, s)
        queries = queries.transpose(1, 2).contigous().view(b * h, t, s)
        values = values.transpose(1, 2).contigous().view(b * h, t, s)
```
As before, the dot products can be computed in a single matrix multiplication, but now between the queries and the keys.


```python
        #
        # Get dot products of queries and keys, and scale
        dot = torch.bmm(queries, keys.transpose(1, 2))
        # -- dot has size (b*h, t, s) containing raw weights

        # scale the dot product
        dot = dot / (k ** (1/2))

        # normalize
        dot = F.softmax(dot, dim=2)
        # - dot now contains row-wise normalized weights
```
We apply the self attention weights dot to the values, giving us the output for each attention head


```python
        #
        # apply the self attention to the values
        out = torch.bmm(dot, values).view(b, h, t, s)
```

To unify the attention heads, we transpose again, so that the head dimension and the embedding dimension are next to each other, and reshape to get concatenated vectors of dimension $e$. We then pass these through the unifyheads layer for a final projection


```python
        #
        # swap h, t back, unify heads
        out = out.transpose(1, 2).contigous().view(b, t, s * h)

        return self.unifyheads(out)
```





### **Building transformer**

#### **The transformer block**

The transformer block applies, in sequence: <font color="green">a self attention layer</font>, <font color="red">layer normalization</font>, <font color="green">a feed forward layer</font>( a single MLP applied independently to each vector), and <font color="red">another layer normalization</font>.<font color="blue">Residual connections</font> are added around both, before the normalization. <br><br>



```python
class TransformerBlock(nn.Module):
    def __init__(self, k, heads):
        super().__init__()

        self.attention = SelfAttention(k, heads=heads)

        self.norm1 = nn.LayerNorm(k)
        self.norm2 = nn.LayerNorm(k)

        self.ff = nn.Sequential(
            nn.Linear(k, 4 * k),
            nn.ReLU(),
            nn.Linear(4 * k, k)
        )

    def forward(self, x):
        attended = self.attention(x)
        x = self.norm1(attented + x)

        fedforward = self.ff(x)
        return self.norm2(fedforward + x)
```




#### **Classification transformer**

The simplest transformer is a *sequence classifier* which takes sentences as input, tokenized into sequence of words and to apply global average pooling to the final output sequence, and to map the result to a softmaxed class vector.(The output sequence is average to produced a single vector representing the whole sequence. This vector is projected down to a vector with one element per class and softmaxed to produce probabilities.)<br><br>
The transformer blocks are permutation equivariant layers, so is the final global average pooling. One way to have the sensitivity to word order is to create a second vector of equal length, that represents the position of the word in the current sentence, and add it to the word embedding.<br>
**position embeddings** We create the embedding vectors like $\mathbf{v}_{12}$ and $\mathbf{v}_{25}$ similar to $\mathbf{v}_{cat}$ and $\mathbf{v}_{dog}$ upto the length of the sequences it can get. It has its drawbacks like to see sequences of every length during training, otherwise the relevant position embeddings don't get trained. It works pretty well and easy to implement.<br>
**position encodings** This is similar to the position embeddings except we don't learn the position vectors, we just choose some function $f: \mathbb{N} \to \mathbb{R}^k$ to map the positions to real values vectors, and let the network figure out how to interpret these encodings. The benefit is that for a well chosen function, the network should be able to dael with sequences that are longer than those its seen in during training. The drawbacks are endcoding function is a complicated hyperparameter, and complicates implementation


```python
#
class Transformer(nn.Module):
    def __init__(self, k, heads, depth, seq_length, num_tokens, num_classes):
        super().__init__()

        self.num_tokens = num_tokens
        self.token_emb = nn.Embedding(num_tokens, k)
        self.pos_emb = nn.Embedding(seq_length, k)

        # the sequence of transformer blocks that does all
        # heavy lifting

        tblocks = []
        for i in range(depth):
            tblocks.append(TransformerBlock(k=k, heads=heads))
        self.tblocks = nn.Sequential(*tblocks)

        # Maps the final output sequence to class logits
        self.toprobs = nn.Linear(k, num_classes)


    def forward(self, x):
        """
        :param x: A (b, t) tensor of integer values representing
                 words (in some predetermined vocabulory).
        :return: A (b, c) tensor of log-probabilities over the
                 classes (where c is the nr. of classes)
        """
        # generate token embeddings
        tokens = self.token_emb(x)
        b, t, k = token_size()

        # generate position embeddings
        positions = torch.arange(t)
        positions = self.pos_emb(positions)[None, :, :].expand(b, t, k)

        x = tokens + positions
        x = self.tblocks(x)

        # Average-pool over the t dimension and project to class
        # probabilities
        x = self.toprobs(x.mean(dim=1))
        return F.log_softmax(x, dim=1)       

```


#### **Text generation transformer**

The trick is try an *autoregressive* model. We'll train a character level transformer to predict the next character in a sequence. We give sequence-to-sequence model a sequence, and we ask it to predict the next character at each point in the sequence. The output of transformer depends on the entire input sequence, so prediction of the next character becomes vacuously easy, just retrieve it from the input. To use self-attention as an autoregressive model, we'll need to ensure that it cann't look forward into the sequence. We do this by applying a mask to the matrix of dot products, before the softmax is applied. This mask disables all elements above the diagonal of the matrix.<br><br>
Masking the self attention, to ensure that elements can only attend to input elemetns that precede them in the sequence by maskingout elements using $-\infty$<br><br>


```python
dot = torch.bmm(queries, keys.transpose(1, 2))

indices = torch.triu_indices(t, t, offset=1)
dot[:, indices[0], indices[1]] = float('-inf')

dot = F.softmax(dot, dim=2)
```
We've handicaped the self-attention module, the model can no longer look forward in the sequence.


#### **The original transformer: encoders and decoders**

The standard structure of sequence-to-sequence models was an encoder-decoder architecture, with **teacher forcing**.  The encoder takes the input sequence and maps it to a latent representation of the whole sequence. This can be either a sequence of latent vectors, or a single one. This vector is then passed to a decoder which unpacks it to the desired target sequence(for instance, the same sentence in another language).<br><br>
*Teacher forcing* refers to the technique of allowing the decoder acces to the input sentence, but in an autoregressive fashion. That is, the decoder generates the output sentence word for word based both on the latent vector and the words it has already generated. This take some of the pressure off the latent representation: the decoder can use word-for-word sampling to take care of the low-level structure like syntax and grammar and use the latent vector to capture more high-level semantic structure. Decoding twice with the same latent represetation would, ideally, give you two different sentences with same meaning.


### **Modern transformers**

In transformer, like BERT and GPT-2, the encoder/decoder configuration was entirely dispensed with. A simple stack of transformer blocks called as decoder-only transformer(an autoregressive model) or encoder-only transformer(for a model without masking).

### **BERT**

BERT is a simple stack of transformer blocks, they are pre-trained on a large general-domain corpus consisting of 800M words from English books(modern work, from unpublished authors), 2.5B words of text from English Wikipedia articles (without markup).<br><br>
Pretraining is done through two tasks:<br>
**Masking** A certain number of words in the input sequence are: masked out, replaced with a random word or kept as is. The model is then asked to predict, for these words, what the original words were. Note that the model doesn't need to predict the entire denoised sentence, just the modified words. Since the model doesn't know which words it will be asked about, it learns a representation for every word in the sequence.<br>
**Next sequence classification** Two sequences of about 256 words are sampled that either (a) follow each other directly in the corpus, or (b) are both taken from random places. The model must then predict whether a or b is the case.<br>
BERT uses WordPiece tokenization, which is somewhere in between word-level and character level sequences. It breaks words like walking up into the tokens walk and ##ing. This allows the model to make some inferences based on word stucture: two verbs ending in -ing have similar grammatical functions, and two verbs starting with walk- have similar semantic function.<br>
The input is prepended with a special <cls> token. The output vector corresponding to this token is used as a sentence representation in sequence classification tasks like next sentence classification (as opposed to the global average pooling over all vectors that we used in our classification model above).<br>
The whole model is then re-trained to finetune the model for the specific task at hand. The authors show that the largest imporovement as compared to previous models comes from bidirectional nature of BERT. That is, prevous models like GPT used an autoregressive mask, which allowed attention only over previous tokens. The fact that in BERT all attention is over the whole sequence is the main cause of the improve performance<br>


#### **GPT-2**

GPT-2  generated sufficiently believable test that large-scale fake news campaigns of the kind seen in the 2016 US presidential election would become effectively a one-person job.<br>
The first trick that authors of GPT-2 employed was to create a new high-quality dataset. To collect more diverse data without sacrificing quality the authors used links posted on the social media site Reddit to gather a large collection of writing with a certain minimum level of social support(expressed on Reddit as Karma)<br>
GPT2 is fundamentally a language generation model, so it uses masked self-attention like we did in our model above. It uses byte-pair encoding to tokenize the language, like the WordPiece encoding breaks words up into tokens that are slightly larget than single character but less than entire words.



#### **Transformer-XL**

The transformer are fundamentally limited by the size of the input. The size of the dot-product matrix grows quadratically in the sequence length, this quickly becomes the bottlenecks as we try to extend the length of the input sequence. Transformer-XL is one of the first successfull transformer model to tackle this problem.<br>
During training, a long sequence of text(longer than the model could deal with) is broken up into shorter segments. Each segment is processed in sequence, with self-attention computed over the tokens in the current segment and the previous segment. Gradients are only computed over the current segment, but information still propagates as the segment window movoes through the text. In theory at layer n, information may be used from n segments ago.<br>
*A similar trick in RNN training is called truncated backpropagation through time. We feed the model a very long sequence, but backpropagate only over part of it. The first part of the sequence, for which no gradients are computed, still influences the values of the hidden states in the part for which they are.*<br>
To make this work, the authors had to let go of the standard position encoding/embedding scheme. Since the position encoding is absolute, it would change for each segment and not lead to a consistent embedding over the whole sequence. Instead they use a relative encoding. For each output vector, a different sequence of position vectors is used that denotes not the absolute position, but the distance to the current output.<br>
This requires moving the position encoding into the attention mechanism. One benefit is that the resulting transformer will likely generalize much better to sequences of unseen length



#### **Sparse transformer**

Sparse transformers tackle the problem of quadratic memeory use head-on. Instead of computing a dense matrix of attention weights(which grows quadratically), they compute the self-attention only for particular pairs of input tokens, resutling in a *sparse* attention matrix, with only $n\sqrt{n}$ explicit elements.<br>
This allows models with very large context sizes, for instance for generative modelling over images, with large dependencies between pixels. The tradeoff is that the sparsity structure is not learned, so by the choice of sparse matrix, we are disabling some interactions between input tokens that might otherwise have been useful. However, two units that are directly related may still interect in higher layers of the  transformer(similar to the way a convolutional net builds upa a larger receptive field with more convolutional layers).<br>
Beyond the simple benefit of training transformers with very large sequence lengths, the sparse transformer also allows a very elegant way of designing an inductive bias. We take out input as a collection of units(words, characters, pixels in an image, nodes in a graph) ans we specify, through the sparsity of the attentin matrix, which units we believe to be related. The rest is just a matter of building the transformer up as deep as it will go and seeing if it tains.

### **Going big**

The big bottleneck in training transformers is the matrix of dot products in self attentin. For a sequence length $t$, this is a dense matrix containing $t^2$ elements. At standard 32-bit precision, and with t = 1000 a batch of 16 such matrices takes up about 64Mb of memory. Since we need to store at least four of them for each head of each self-attentin operation (before and after softmax, plus their gradients), that limits us to at most twelve 4-head layers in a standard 12Gb GPU. In practice, we get evenless, since the inputs and outputs also take up a lot of memory(although the dot product dominate)<br>
And yet models reported in the literature contain much longer sequence lengths, with 48 or more layers, using dense dot product matrices. These models are trained on clusters, of course, but a single GPU is still required to do a single forward/backward pass. How do we fit such humongous transformers into 12Gb of memory?There are three main tricks:<br><br>
**Half precision** On modern GPUs and on TPUs, tensor computations can be done efficiently on 16-bit float tensors. This isn't quite as simple as just setting the dtype of the tensor to $torch.float16$. For some parts of the network, like the loss, 32 bit precision is required. But most of this can be handled with relative ease by existing librries. Practically, this doubles your effective memory.<br>
**Gradient accumulation** For a large model, we may only be able to perform a forward/backward pass on a single instance. Batch size 1 is not likely to lead to stable learning. Luckily, we can perform a single forward/backward for each instance in a larger batch, and simply sum the gradients we find (this is a consequence of the multivariate chain rule). When we hit the end of the batch, we do a single step of gradient descent, and zero out the gradient. <br>
**Gradient checkpointing** If your model is so big that even a single forward/backward won't fit in memory, you can trade off even more computation for memory efficiency. In gradient checkpointing, you separate your model insto sections. For each section, you do a separate forward/backward to compute the gradients, without retaining the intermediate values for the rest.


### **Conclusion**

1.  **The current performance limit is purely in the hardware**. Unlike convolutions or LSTMs the current limitations to what they can do are entirely determined by how big a model we can fit in GPU memory and how much data we can push through it in a reasonanle amoutn of time. I have no doubt, we will eventually hit the point where more layers and more data won't help anymore, but we don't seem to have reached that point yet.
2.  **Transformers are extremely generic**. So far, the big successes have been in language modelling, with some more modest achievements in image and music analysis, but the transformer has a level of generality that is waiting to be exploited. The basic transformer is a set-to-set model. So long as your data is a set of units, you can apply a transformer. Anything else you know about your data (like local structure) you can add by means of position embeddings, or by manipulating the structure of the attention matrix (making it sparse, or masking out parts)<br><br>



**References:**
1.   https://peterbloem.nl/blog/transformers
2.   